In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

## Read grid info

In [2]:
# Create buses
num_buses = 2848
buses = np.arange(1, num_buses+1)

# Read gen buses
file_path = f'../../data/IEEE_Case2848/zones/gen_bus.csv'
gen_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read thermal buses
file_path = f'../../data/IEEE_Case2848/zones/thermal_bus.csv'
thermal_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read wind gen buses
file_path = f'../../data/IEEE_Case2848/zones/wind_bus.csv'
wind_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read load buses
file_path = f'../../data/IEEE_Case2848/zones/load_bus.csv'
load_buses = pd.read_csv(file_path, header=None, index_col=None)

In [3]:
# Boolean index of thermal buses
thermal_bidx = np.isin(buses, thermal_buses)

# Boolean index of wind buses
wind_bidx = np.isin(buses, wind_buses)

# Boolean index of load buses
load_bidx = np.isin(buses, load_buses)

In [4]:
# Time steps
nt = 12

# Read wind and load inputs into MATPOWER
num_samples = 1000

max_num = 1528

## Model inputs

In [ ]:
# Empty list to store data
x = []

# Empty node feature matrix
node_features = np.zeros((buses.shape[0], 2*nt))

# Assign wind and load features
for i in range(max_num):
    # File path
    wind_path = f'../../data/IEEE_Case2848/inputs/wind/sample_{i+1}.csv'
    load_path = f'../../data/IEEE_Case2848/inputs/load/sample_{i+1}.csv'

    # Read wind and load
    try:
        wind = pd.read_csv(wind_path, header=None, index_col=None)
        load = pd.read_csv(load_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    node_features[wind_bidx, :nt] = wind.values
    node_features[load_bidx, nt:2*nt] = load.values

    x.append(node_features.copy())

## PB ground truth

In [ ]:
y = []

# Add zeros to make the dimension as [num_buses, nt*2]
# all_PG = np.zeros((num_buses, nt*2))
PB = np.zeros((num_buses, nt))

temp1 = np.zeros((num_buses, nt))
temp2 = np.zeros((num_buses, nt))
temp3 = np.zeros((num_buses, nt))

for i in range(max_num):
    wind_path = f'../../data/IEEE_Case2848/outputs/deployed_wind/sample_{i+1}.csv'
    PG_path = f'../../data/IEEE_Case2848/outputs/PG/sample_{i+1}.csv'
    DL_path = f'../../data/IEEE_Case2848/outputs/deployed_load/sample_{i+1}.csv'
    try:
        wind = pd.read_csv(wind_path, header=None, index_col=None)
        PG = pd.read_csv(PG_path, header=None, index_col=None)
        DL = pd.read_csv(DL_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    temp1[wind_bidx] = wind.values
    temp2[thermal_bidx] = PG.values
    temp3[load_bidx] = DL.values

    PB = temp1 + temp2 - temp3

    y.append(PB.copy())

## Edge index and attributes

In [11]:
# Read branch info
file_path = f'../../data/IEEE_Case2848/branch_params/branch_params.csv'
branch = pd.read_csv(file_path, header=None, index_col=None).to_numpy()

# Get branch index and attr
edge_index = branch[:, :2] - 1

# Convert to standard format
edge_index = torch.tensor(edge_index.T, dtype=torch.long)

## SAGE dataset

In [ ]:
# Check the status of old dataset
dir = f'../../train_val_test_dataset/IEEE_Case2848/PB/processed'
if not os.path.exists(dir):
    print(f'There is no dataset found!')
else:
    shutil.rmtree(dir)
    print(f'The old dataset has been deleted!')

# Save new dataset
data_list = []
for i in range(len(x)):
    X = torch.from_numpy(x[i]).float()
    Y = torch.from_numpy(y[i]).float()
    graph = Data(x=X, y=Y, edge_index=edge_index)
    data_list.append(graph)

from CustomDataset import CustomDataset
# Not that if there is already saved dataset, this cell won't work
# Save train, val and test data
root = f'../../train_val_test_dataset/IEEE_Case2848/PB/'
CustomDataset(root=root, data_list=data_list)